### Про метрику

Важной частью этого чекпоинта является выбор метрики. Напомним, что на текущий момент видение проекта - LLM / Agent-based ассистент для задач ментального здоровья. То есть в качестве стандартных NLP-метрик необходимо использовать метрики для задачи Language Modeling / текстовой генерации, такие как вывод обученной reward-модели, perplexity, side-by-side сравнение на асессорах, и так далее. При этом для ML-бейзлайна на текущем этапе мы будем решать задачу текстовой классификации. Для этой задачи мы можем использовать классические метрики для текстовой классификации, такие как accuracy, precision, recall, F1-score, и так далее. При этом у нас нет негативых сэмплов, так как каждое наблюдение принадлежит какому-то классу, связанного с ментальным здоровьем. Поэтому будем использовать просто accuracy (сбалансированную по классам, так как датасет является не супер сбалансированным)

In [1]:
import pandas as pd
import numpy as np

In [2]:
# нанов в процентах немного, а данных много, поэтому позволим себе просто удалить наны на текущем этапе
reddit_df = pd.read_csv('../data/reddit_mental_health_posts_preprocessed.csv')
reddit_df = reddit_df[['title', 'body', 'subreddit']]
reddit_df = reddit_df.rename(columns={'subreddit': 'target'})
print('len of reddit_df: ', len(reddit_df))
reddit_df = reddit_df.dropna()#.sample(1000)
print('len of reddit_df after dropna: ', len(reddit_df))
reddit_df.head(5)

len of reddit_df:  151288
len of reddit_df after dropna:  148936


,title,body,target
0,get extremely anxious ’ working 247,month ago accepted full time software engineer...,ADHD
1,cant clean house feel incredibly motivated cle...,hey guy curious anyone else issue apartment fu...,ADHD
2,need help,6 exam next 2 week one monday havent studied f...,ADHD
3,anyone chat,anyone struggling addadhd ’ interesting chatti...,ADHD
4,figuring eat suck,whenever get hungry never eat dont know eat en...,ADHD


In [3]:
# видно, что датасет не супер сбалансирован
reddit_df.target.value_counts()

target
OCD           41812
ADHD          37058
depression    23770
ptsd          23758
aspergers     22538
Name: count, dtype: int64

### Соединим title и body, и построим эмбеддинги

Для сравнения будем использовать:

+ tf-idf

+ tf-idf + SVD разложение (понижение размерности)

+ Word2Vec с усреднением эмбеддингов по словам

+ Простенький трансформер

In [4]:
reddit_df['text'] = 'Title: ' + reddit_df['title'] + '; Body: ' + reddit_df['body']
reddit_df = reddit_df.drop(['title', 'body'], axis=1)
reddit_df.head(5)

,target,text
0,ADHD,Title: get extremely anxious ’ working 247; Bo...
1,ADHD,Title: cant clean house feel incredibly motiva...
2,ADHD,Title: need help; Body: 6 exam next 2 week one...
3,ADHD,Title: anyone chat; Body: anyone struggling ad...
4,ADHD,Title: figuring eat suck; Body: whenever get h...


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
from gensim.models import Word2Vec

In [6]:
def create_embeddings(df):
    results = []
    df1 = df.copy()
    df2 = df.copy()
    df3 = df.copy()
    df4 = df.copy()
    # Let's start with tf-idf embeddings
    print("Creating tf-idf embeddings")
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X_tfidf = tfidf_vectorizer.fit_transform(df1['text']).toarray()
    results.append((df1, "tf-idf", X_tfidf))
    
    # Now let's add SVD to serve as dimensionality reduction
    print("Creating tf-idf + SVD embeddings")
    svd = TruncatedSVD(n_components=300, random_state=42)
    X_tfidf_svd = svd.fit_transform(X_tfidf)
    results.append((df2, "tf-idf+SVD", X_tfidf_svd))
    
    # Time to do something more advanced - shall we use transformers?
    print("Creating Sentence-BERT embeddings")
    model = SentenceTransformer('all-MiniLM-L6-v2') # something lightweight so that the laptop does not die
    sentences = df3['text'].tolist()
    X_sbert = model.encode(sentences, batch_size=32, show_progress_bar=True)
    results.append((df3, "Sentence-BERT", X_sbert))
    
    # Word2Vec with averaging is not dead, is it?
    print("Creating Word2Vec embeddings")
    tokenized_texts = [word_tokenize(text.lower()) for text in df4['text']]
    w2v_model = Word2Vec(
        sentences=tokenized_texts,
        vector_size=128,
        window=5,
        min_count=5,
        workers=4,
        seed=42
    )
    # Averaging word vectors
    doc_vectors = []
    for doc in tokenized_texts:
        word_vecs = [w2v_model.wv[word] for word in doc if word in w2v_model.wv]
        if len(word_vecs) > 0:
            doc_vectors.append(np.mean(word_vecs, axis=0))
        else:
            doc_vectors.append(np.zeros(w2v_model.vector_size))
    X_word2vec = np.array(doc_vectors)
    results.append((df4, "Word2Vec", X_word2vec))
    
    print('Success')
    return results

In [7]:
embedding_df = create_embeddings(reddit_df)

Creating tf-idf embeddings
Creating tf-idf + SVD embeddings
Creating Sentence-BERT embeddings


Batches:   0%|          | 0/4655 [00:00<?, ?it/s]

Creating Word2Vec embeddings
Success


### Начнем моделировать. Для каждого из эмбеддингов построим 4 модели:

+ Logistic Regression

+ KNN

+ CatBoost

+ MLP (простой MLP из sklearn)

После этого измерим качество моделей на тестовой выборке и выберем лучшую. Будем выбирать по balanced accuracy, а выборку будем делить на трейн и тест в соотношении 90:10 (для CatBoost и MLP будем использовать дополнительную валидационную выборку)

In [8]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score, precision_score, recall_score, confusion_matrix

### CatBoost GridSearch

In [ ]:
results = []
for max_depth in range(4, 8):
    for iterations in range(500, 1051, 100):
        model = CatBoostClassifier(
            iterations=iterations,
            max_depth=max_depth,
            random_state=42,
            verbose=False,
            early_stopping_rounds=20
        )
        
        model.fit(X_train, y_train, eval_set=(X_val, y_val))
        
        y_pred = model.predict(X_val)
        
        accuracy = round(balanced_accuracy_score(y_val, y_pred), 3)
        precision = round(precision_score(y_val, y_pred, average='micro'), 3)
        recall = round(recall_score(y_val, y_pred, average='micro'), 3)
        
        results.append(
            ((iterations, max_depth), (accuracy, precision, recall))
        )
        
        print((iterations, max_depth), (accuracy, precision, recall))

In [ ]:
sorted(results, key=lambda val: val[1][2])[-3:]

#### Итого, лучшие гиперпараметры - 900 деревьев глубиной до 7

P.S. Отбор происходил по recall, потому что в нашей задаче будто бы цена FN ошибки намного больше, чем FP. Лучше человека случайно причислить к больным и отправить на доп. обследования, чем объявить здоровым несмотря на болезнь. 

In [9]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"


In [10]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, embedding_name, model_name, 
                             X_val=None, y_val=None):
    
    if model_name == "Logistic Regression":
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    elif model_name == "CatBoost":
        if X_val is not None and y_val is not None:
            model = CatBoostClassifier(
                iterations=900,
                max_depth=7,
                random_state=42,
                verbose=False,
                early_stopping_rounds=20
            )
            model.fit(X_train, y_train, eval_set=(X_val, y_val))
        else:
            model = CatBoostClassifier(random_state=42, verbose=False)
            model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    elif model_name == "KNN":
        model = KNeighborsClassifier(n_neighbors=16) # 2 ** 4
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
    elif model_name == "MLP":
        model = MLPClassifier(
            hidden_layer_sizes=(128, 64),
            random_state=42,
            early_stopping=True,
            validation_fraction=0.05,
            max_iter=100
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    accuracy = balanced_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred)
    return accuracy, precision, recall, conf_matrix

In [11]:
models = ["KNN", "Logistic Regression", "CatBoost", "MLP"]
results = []

for df_copy, embedding_name, X_embeddings in embedding_df:
    if embedding_name == "tf-idf": # неудачная размерность 5000, все слишком долго обучалось и я решил выкинуть обычный tf-idf из анализа
        continue
    print(f"Processing {embedding_name}")
    y = df_copy['target'].values
    X_temp, X_test, y_temp, y_test = train_test_split(
        X_embeddings, y, test_size=0.1, random_state=42, stratify=y
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.06, random_state=42, stratify=y_temp
    )
    print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")
    val_merged = False
    for model_name in models:
        print("="*30)
        print(f"Training {model_name}")
        if model_name in ["CatBoost"]:
            accuracy, precision, recall, conf_matrix = train_and_evaluate_models(
                X_train, X_test, y_train, y_test, embedding_name, model_name, X_val, y_val
            )
        else:
            if not val_merged:
                X_train = np.concatenate([X_train, X_val], axis=0)
                y_train = np.concatenate([y_train, y_val], axis=0) # объединяем трейн и валидацию для всех моделей кроме СB
                val_merged = True

            accuracy, precision, recall, conf_matrix = train_and_evaluate_models(
                X_train, X_test, y_train, y_test, embedding_name, model_name
            )
        results.append({
            'Embedding': embedding_name,
            'Model': model_name,
            'Balanced_Accuracy': accuracy,
            'Weighted_Precision': precision,
            'Weighted_Recall': recall,
            'Confusion_Matrix': conf_matrix
        })
        print(f"{embedding_name} + {model_name}")
        print(f"Acc: {accuracy:.4f}, Pr:{precision:.4f}, Rec:{recall:.4f}")
        print(conf_matrix)

Processing tf-idf+SVD
Train shape: (125999, 300), Val shape: (8043, 300), Test shape: (14894, 300)
Training KNN
tf-idf+SVD + KNN
Acc: 0.5834, Pr:0.6242, Rec:0.6166
[[2619  339  490  176   82]
 [ 254 3201  332  249  145]
 [ 378  326 1279  196   75]
 [ 386  404  437 1018  132]
 [ 287  433  356  233 1067]]
Training Logistic Regression
tf-idf+SVD + Logistic Regression
Acc: 0.7473, Pr:0.7692, Rec:0.7684
[[3097  158  218  172   61]
 [ 148 3556  132  229  116]
 [ 310  206 1472  205   61]
 [ 218  244  140 1668  107]
 [ 140  254  100  230 1652]]
Training CatBoost
tf-idf+SVD + CatBoost
Acc: 0.7454, Pr:0.7677, Rec:0.7667
[[3069  156  227  188   66]
 [ 119 3570  142  233  117]
 [ 284  185 1468  235   82]
 [ 196  211  148 1695  127]
 [ 143  245  124  247 1617]]
Training MLP
tf-idf+SVD + MLP
Acc: 0.7566, Pr:0.7750, Rec:0.7761
[[3093  123  233  164   93]
 [ 114 3570  118  194  185]
 [ 267  152 1506  195  134]
 [ 182  217  154 1603  221]
 [ 102  194   92  201 1787]]
Processing Sentence-BERT
Train shap

In [12]:
results_df = pd.DataFrame(results)

In [13]:
results_df.pivot_table(
        index='Embedding', 
        columns='Model', 
        values='Weighted_Recall'
)

Model,CatBoost,KNN,Logistic Regression,MLP
Embedding,,,,
Sentence-BERT,0.769840,0.756278,0.765409,0.784007
Word2Vec,0.721364,0.634752,0.710420,0.738351
tf-idf+SVD,0.766685,0.616624,0.768430,0.776084


In [14]:
results_df.pivot_table(
        index='Embedding', 
        columns='Model', 
        values='Balanced_Accuracy'
)

Model,CatBoost,KNN,Logistic Regression,MLP
Embedding,,,,
Sentence-BERT,0.750471,0.730904,0.746113,0.767682
Word2Vec,0.698627,0.600103,0.688691,0.716208
tf-idf+SVD,0.745381,0.583416,0.747252,0.756617


In [15]:
results_df.pivot_table(
        index='Embedding', 
        columns='Model', 
        values='Weighted_Precision'
)

Model,CatBoost,KNN,Logistic Regression,MLP
Embedding,,,,
Sentence-BERT,0.770112,0.762946,0.765976,0.788205
Word2Vec,0.720500,0.631221,0.709199,0.738150
tf-idf+SVD,0.767727,0.624210,0.769195,0.774979


In [16]:
results_df.to_csv('reddit_baseline_results.csv')

### Выводы

+ MLP (на удивление) оказалась и самой сильной, и самой устойчивой моделью. В отличие от всех других моделей, здесь качество (в виде balanced accuracy) на тестовой выборке не опустилось ниже 0.7

+ KNN проигрывает всем моделям на всех эмбеддингах, единственный показал качество ниже 0.6. Это значительно хуже других моделей

+ Он же (KNN) оказался самым неустойчивым, где разброс между лучшей и худшей моделью превышает 0.1 (во всех остальных моделях спред в районе 0.05)

+ Несмотря на кардинально разные алгоритмы и структуры моделей, CatBoost и LogReg ведут себя удивительно схоже с точки зрения качества

+ Хоть мы и обучили немного всего, это честный труд :) А еще мы получили на удивление действительно интересные результаты

### Возможный (и желательный) TODO:

+ DONE - Пофиксить логику с валидационной выборкой. Пусть она создается не везде, а только там, где нужно, чтобы не терять данные

+ DONE - Измерить больше метрик, построить confusion матрицы

+ Выбрать лучшую модель (например, CatBoost / MLP на Sentence-BERT эмбеддингах), потюнить ее гиперпараметры, и посмотреть, как будет меняться качество. Выбрать лучшие гиперпараметры, и замерить финальное качество